In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
datasetPath = "./drive/MyDrive/dataset/"
parameterPath = "./drive/MyDrive/parameters/"

In [3]:
import torch
import torch.nn as nn
#import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader
import numpy as np
# import torch.nn.functional as F

In [13]:
# Training dataset 다운로드
cifar10_train = dataset.CIFAR10(root = datasetPath, # 데이터셋이 저장된 위치
                                train = True,
                                transform = transform.ToTensor(),
                                download = True)
# Testing dataset 다운로드
cifar10_test = dataset.CIFAR10(root = datasetPath,
                               train = False,
                               transform = transform.ToTensor(),
                               download = True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
cfg = {
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg['VGG13'])
        self.classifier = nn.Linear(512, 256)
        self.classifier2 = nn.Linear(256, 10)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.bn1(self.classifier(out))
        out = self.relu(self.classifier2(out))
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [8]:
class VGG (nn.Module):
  def __init__(self): # Parameter를 가지는 lalyer 정보
    super(VGG, self).__init__()  #자기 자신을 또 부름
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.conv7 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
    self.conv8 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.fc1 = nn.Linear(in_features=512, out_features=256)
    self.fc2 = nn.Linear(in_features=256, out_features=10)
    self.relu = nn.ReLU()
    self.bn1d = nn.BatchNorm1d(256)
    self.bn2d_1 = nn.BatchNorm2d(64)
    self.bn2d_2 = nn.BatchNorm2d(128)
    self.bn2d_3 = nn.BatchNorm2d(256)
    self.bn2d_4 = nn.BatchNorm2d(512)
    self.avgPool2d1 = nn.AvgPool2d(kernel_size=1, stride=1)  # ReLU, Pooling layer는 parameter를 안 가짐
    self.avgPool2d2 = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self, x):  # 모델의 동작 순서, 각 layer & function의 입출력 관계
    out = self.relu(self.bn2d_1(self.conv1(x)))
    out = self.relu(self.bn2d_1(self.conv2(out)))
    out = self.avgPool2d2(out)
    out = self.relu(self.bn2d_2(self.conv3(out)))
    out = self.relu(self.bn2d_2(self.conv4(out)))
    out = self.avgPool2d2(out)
    out = self.relu(self.bn2d_3(self.conv5(out)))
    out = self.relu(self.bn2d_3(self.conv6(out)))
    out = self.avgPool2d2(out)
    out = self.relu(self.bn2d_4(self.conv7(out)))
    out = self.relu(self.bn2d_4(self.conv8(out)))
    out = self.avgPool2d2(out)
    out = self.relu(self.bn2d_4(self.conv9(out)))
    out = self.relu(self.bn2d_4(self.conv10(out)))
    out = self.avgPool2d2(out)
    out = self.avgPool2d1(out)

    out = out.view(out.size(0), -1) # feature map 평탄화
    out = self.bn1d(self.fc1(out))
    out = self.relu(self.fc2(out))
    return out

In [ ]:
training_epoch = 20
batch_size = 100
learning_rate = 0.1
loss_function = nn.CrossEntropyLoss()
network = VGG("cfg")
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)

data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [ ]:
network.train()
network = network.to('cuda:0')
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = len(data_loader)

    for img, label in data_loader:
        
        img = img.to('cuda:0')
        label = label.to('cuda:0')
        pred = network(img)
        loss = loss_function(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')          

Epoch: 1 Loss = 0.014199
Epoch: 2 Loss = 0.014513
Epoch: 3 Loss = 0.010438
Epoch: 4 Loss = 0.006648
Epoch: 5 Loss = 0.009584
Epoch: 6 Loss = 0.012463
Epoch: 7 Loss = 0.012813
Epoch: 8 Loss = 0.009591
Epoch: 9 Loss = 0.007551
Epoch: 10 Loss = 0.004110
Epoch: 11 Loss = 0.003328
Epoch: 12 Loss = 0.001967


In [ ]:
torch.save(network.state_dict(), parameterPath+"VGG_cifar10.pth")

In [ ]:
network.eval()
network = network.to('cpu')
img_test = torch.tensor(np.transpose(cifar10_test.data , (0, 3, 1, 2))) /255.
label_test = torch.tensor(cifar10_test.targets)

with torch.no_grad(): # test에서는 기울기 계산 제외
    prediction = network(img_test) # 전체 test data를 한번에 계산

correct_prediction = torch.argmax(prediction, 1) == label_test
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy.item())